# Builds table with exposure data for a directory tree

In [1]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import exifread
import rawpy
from astropy.table import Table

Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_exif(dirpath):
    table = Table(names=('File', 'ISO', 'Exptime'), dtype=('S', 'S', 'S'))

    list_files = glob.glob(dirpath + '/*.ARW')
    for filename in list_files:
        
        f = open(filename, 'rb')
        tags = exifread.process_file(f)
        iso = ''
        exptime = ''
        for tag in tags.keys():
            if tag in ['EXIF ExposureTime']:
                exptime = tags[tag]
            if tag in ['EXIF ISOSpeedRatings']:
                iso = tags[tag]

        table.add_row((os.path.basename(filename), iso, exptime))    

    table.sort('File')

    return table

In [3]:
basepath = '../astrophotography_data/cls/'
dirpath = os.path.join(basepath,'with_filter')
t = get_exif(dirpath)
t.pprint_all()

     File      ISO  Exptime
-------------- ---- -------
I1600_f2.8.ARW 1600      30
  I1600_f2.ARW 1600      30
  I1600_f4.ARW 1600      30
 I200_f2.8.ARW  200      30
   I200_f2.ARW  200      30
   I200_f4.ARW  200      30
 I400_f2.8.ARW  400      30
   I400_f2.ARW  400      30
   I400_f4.ARW  400      30
 I800_f2.8.ARW  800      30
   I800_f2.ARW  800      30
   I800_f4.ARW  800      30


In [4]:
basepath = '../astrophotography_data/cls/'
dirpath = os.path.join(basepath,'without_filter')
t = get_exif(dirpath)
t.pprint_all()

     File      ISO  Exptime
-------------- ---- -------
I1600_f2.8.ARW 1600      30
  I1600_f4.ARW 1600      30
 I200_f2.8.ARW  200      30
   I200_f4.ARW  200      30
 I400_f2.8.ARW  400      30
   I400_f4.ARW  400      30
   I800_f4.ARW  800      30
